In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 100

import seaborn as sns
import pandas_profiling as pdp
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import collections
import re
import feather
import codecs

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

In [97]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')
df_all=pd.concat([train,test],axis=0,sort=False).reset_index(drop=True)
df_all.columns=['id','rent','address','access','floor_info','old','direction','square','floor','bath','kitchen','broadcast','facility','parking','neighbors','structure','period']

train_feat1=pd.read_feather('feature1/data/train_feat1.ftr')
test_feat1=pd.read_feather('feature1/data/test_feat1.ftr')

In [98]:
# 外れ値、異常データを削除

def drop_train_outlier(df_train):
    train_feat1=pd.read_feather('feature1/data/train_feat1.ftr')
    train=df_train[df_train['id']!=20428][df_train['id']!=20232][df_train['id']!=20927]
    train_feat1=train_feat1[train_feat1['id']!=20428][train_feat1['id']!=20232][train_feat1['id']!=20927]

# 面積に対する外れ値を取得
    index1=train_feat1.query("rent>1000000 & square_num<100").index
    index2=train_feat1.query("rent>1500000").index
    index3=train_feat1.query("square_num>400").index
    index4=train_feat1.query('rent>100000 & square_num<10').index
    
# 築年数に対する外れ値を取得
    index5=train_feat1.query("old_num>1000").index
    
    drop_index=np.concatenate([index1,index2,index3,index4,index5])
    
    train_drop=train.drop(drop_index).reset_index(drop=True)
    train_feat1_drop=train_feat1.drop(drop_index).reset_index(drop=True)
    
    return train_drop

# テストデータ中の外れ値を補正
def modify_test_outlier(df_test):
    test_feat1=pd.read_feather('feature1/data/test_feat1.ftr')

    df_test.loc[test_feat1.query('old_num>1000').index,'築年数']='11年4ヶ月'
    return df_test
    
train_drop=drop_train_outlier(train)
test_mod=modify_test_outlier(test)

In [99]:
train_index=len(train_drop)
test_index=len(test_mod)

df_all=pd.concat([train_drop,test_mod],axis=0,sort=False).reset_index(drop=True)
df_all.columns=['id','rent','address','access','floor_info','old','direction','square','floor','bath','kitchen','broadcast','facility','parking','neighbors','structure','period']

In [5]:
train_feat3=pd.read_feather('feature3/data/train_feat3.ftr')
test_feat3=pd.read_feather('feature3/data/test_feat3.ftr')

## 最寄り駅までアクセスを距離に換算する
 バスの移動速度を11km/h=183m/分  
 https://www.mlit.go.jp/road/ir/ir-data/jroad04/01-06.html  
 徒歩80m/分,車400m/分  
 https://www.id-home.net/knowledge/%E4%B8%8D%E5%8B%95%E7%94%A3%E3%81%AE%E5%BE%92%E6%AD%A9%E6%99%82%E9%96%93%E3%81%A8%E8%A1%A8%E7%A4%BA%E5%9F%BA%E6%BA%96%E3%82%92%E8%A7%A3%E8%AA%AC%EF%BC%81/

In [6]:
# 最寄りの駅情報を４つに分ける
def split_access(x):
    access_list=x.split('\t\t')
    num_list=len(access_list)

    if num_list==1:
        return access_list[0],'nan','nan','nan'
    elif num_list==2:
        return access_list[0],access_list[1],'nan','nan'
    elif num_list==3:
        return access_list[0],access_list[1],access_list[2],'nan'
    else:
        return access_list[0],access_list[1],access_list[2],access_list[3]

# 各最寄り駅情報を路線、駅、移動方法に分ける
def access_enc(x,func):
    if x==0:
        return 'nan'
    else:
        split_list=x.split('\t')
    #  路線名を取得
        if func=='line':
            if '線' in split_list[0]:
                return split_list[0]
            else:
                return 'nan'

        # 駅名を取得
        elif func=='station':
            if len(split_list)>=2 and '駅' in split_list[1]: 
                return split_list[1]
            else:
                return 'nan'

        # 距離を取得
        else:
            if len(split_list)==3:
                    return split_list[2]
            else:
                return 'nan'
        
#  距離を数値型に変更
def enc_num(x):

    dist=0
    
    walk = re.findall(r'徒歩.+?分', x)
    if len(walk)!=0:
        walk_t=int(walk[0][2:-1])
        dist+=80*walk_t
        
    bus = re.findall(r'バス\(.+?\)', x)
    if len(bus)!=0:
        bus_t=int(bus[0][3:-2])
        dist+=183*bus_t

    
    car = re.findall(r'車.+?km', x)
    if len(car)!=0:
        car_m=float(car[0][1:-2])*1000
        dist+=car_m
    
# 欠損の場合は-999をいれる
    if dist==0:
        return -999
    else:
        return dist


tmp=df_all['access'].apply(lambda x:split_access(x))

# 最寄り駅、二つまで取得する。
for i in range(2):
    df_all['access_'+str(i+1)]=tmp.apply(lambda x:x[i])
    df_all['access_'+str(i+1)+'_line']=df_all['access_'+str(i+1)].apply(lambda x:access_enc(x,'line'))
    df_all['access_'+str(i+1)+'_station']=df_all['access_'+str(i+1)].apply(lambda x:access_enc(x,'station'))
    df_all['access_'+str(i+1)+'_distance']=df_all['access_'+str(i+1)].apply(lambda x:access_enc(x,'distance'))
    df_all['access_'+str(i+1)+'_distance']=df_all['access_'+str(i+1)+'_distance'].apply(lambda x:enc_num(x))

# 数値型にエンコード

for col in ['line','station']:
    le=LabelEncoder()
    encoder=list(set(df_all['access_1_'+col].unique()) | set(df_all['access_2_'+col].unique())) 
    
    le.fit(encoder)
    for i in range(2):
        df_all['access_'+str(i+1)+'_'+col]=le.transform(df_all['access_'+str(i+1)+'_'+col])
        df_all['access_'+str(i+1)+'_'+col]=df_all['access_'+str(i+1)+'_'+col].astype('category')


## 地下室含めた、階の総数を加える

In [7]:
def room_floor(src):
    tmp=src.split('／')
    # floor_infoは２分割までできるので、分割できる場合は、一つ目の要素が部屋の階
    if len(tmp)==2:
        return tmp[0]
    
    if len(tmp)==1:
        if '階建' in tmp[0]:
             return 'nan'
        else:
            return tmp[0]

def room_floor_enc(x):
    #  数字部分の抜き出し
    if x=='nan':
        return 'nan'
    elif x=='':
        return 'nan'
    
    else:

        num=re.findall(r'\d+',x)[0]
    #  地下の場合はマイナスにする
    if '地下' in x:
        return -1*int(num)
    else:
        return int(num)
    
 #  部屋の階を数値化
df_all['floor']=df_all['floor'].fillna('nan')        
df_all['room_floor']=df_all['floor'].apply(lambda x:room_floor(x))
df_all['room_floor']=df_all['room_floor'].apply(lambda x:room_floor_enc(x))

In [8]:
def building_floor(src):
    tmp=src.split('／')
    # floor_infoは２分割までできるので、分割できる場合は、一つ目の要素が部屋の階
    if len(tmp)==2:
        return tmp[1]
    
    if len(tmp)==1:
        if '階建' in tmp[0]:
             return tmp[0]
        else:
            return 'nan'

# 何階建かを取得
def building_floor_enc(x):
    if x=='nan':
        return -999
        
    else:
        return int(x.split('階建')[0])
    
# 　地下の階数を取得
def check_underground(x):
# 欠損部分は、0を埋める
    if x=='nan':
        return -999
    else:
        under_info=re.findall('地下+\d',x)
        if len(under_info)==0:
            return 0
        else:
            return int(re.findall('地下+\d',x)[0][-1])
    
df_all['floor']=df_all['floor'].fillna('nan')        
df_all['building_floor']=df_all['floor'].apply(lambda x:building_floor(x))

# 地下特徴量を加える
df_all['underground']=0
df_all['underground']=df_all['building_floor'].apply(lambda x:check_underground(x))

# 建物の階数を加える
df_all['building_floor']=df_all['building_floor'].apply(lambda x:building_floor_enc(x))

df_all['total_floor']=df_all['building_floor']+df_all['underground']

## 駐車場、駐輪場、バイク置き場の情報を追加する。

In [44]:
def add_parking_info(x,park_type):

    if x=='nan':
        return 'nan','nan'
    else:
        text=x
    park=x.split('\t')

    if park_type=='駐車場':
        car_info='nan'
        car_price='nan'
        if '駐車場' in text:
            try:
                car_idx=park.index('駐車場')
                car_info=park[car_idx+1]
            except ValueError:
                return 'nan','nan'
                
            try:
                if '円' in park[car_idx+2]:
                    car_price=park[car_idx+2]
                else:
                    car_price='nan'
            except IndexError:
                car_price='nan'

        return car_info,car_price


    if park_type=='駐輪場':
        bicycle_info='nan'
        bicycle_price='nan'
        if '駐輪場' in text:
            try:
                bicycle_idx=park.index('駐輪場')
                bicycle_info=park[bicycle_idx+1]
            except ValueError:
                return 'nan','nan'
            
            try:
                if '円' in park[bicycle_idx+2]:
                    bicycle_price=park[bicycle_idx+2]
                else:
                    bicycle_price='nan'
            except IndexError:
                bicycle_price='nan'

        return bicycle_info,bicycle_price
    
    if park_type=='バイク置き場':
        bike_info='nan'
        bike_price='nan'
        if 'バイク置き場' in text:
            try:
                bike_idx=park.index('バイク置き場')
                bike_info=park[bike_idx+1]
            except ValueError:
                return 'nan','nan'
            
            try:
                if '円' in park[bike_idx+2]:
                    bike_price=park[bike_idx+2]
                else:
                    bike_price='nan'
            except IndexError:
                bike_price='nan'
                
        return bike_info,bike_price
    
# 駐車場があるかどうか０、１分類
def add_is_park(x):
    if x=='nan':
        return 'nan'
    
    elif x=='無':
        return 0
    
    else:
        return 1

# 住居に備え付けの駐車場があるかどうか、0,1分類
def add_house_park(x):
    if x=='nan':
        return 'nan'
    
    elif '空有' in x or '空無' in x:
        return 1
    
    else:
        return 0
    
def add_other_park(x):
    if x=='nan':
        return 'nan'
    
    elif '近隣' in x:
        return 1
    
    else:
        return 0
    
def add_park_num(x):
# 台数が書いてない場合は１台にする
    if x=='空有' or x=='近隣':
        return 1

    elif x=='空無':
        return 0
    
    elif len(re.sub(r'\D', '', x))!=0:
        return int(re.sub(r'\D', '', x))
    
    elif x=='nan':
        return 'nan'
    
    else:
        return 0
        
def add_park_price(x):
    if x=='nan':
        return 'nan'
    
    price=int(re.sub(r'\D', '',x ))
    if '税抜' in x:
        price=price*1.08

    return price
        
    
def modify_outlier_price(df):
    index1=df.query('parking_price_car!="nan"').query('parking_price_car>100000').index
    df.loc[index1,'parking_price_car']='nan'
    index2=df.query('parking_price_bicycle!="nan"').query('parking_price_bicycle>20000').index
    df.loc[index2,'parking_price_bicycle']='nan'
    index3=df.query('parking_price_bike!="nan"').query('parking_price_bike>30000').index
    df.loc[index3,'parking_price_bike']='nan'
    return df


# 欠損を埋める
df_all['parking']=df_all['parking'].fillna('nan')

for v,p in zip(['car','bicycle','bike'],['駐車場','駐輪場','バイク置き場']):
    parking=df_all['parking'].apply(lambda x :add_parking_info(x,p)) 
    df_all[v+'_info']=parking.apply(lambda x:x[0])
    df_all[v+'_price']=parking.apply(lambda x:x[1])
    df_all.loc[df_all[df_all['car_price']=='(自転車駐輪相談可能（空き要確認）6，000円を1年分一括徴収（非課税）/原付駐輪は現在満車)'].index,'car_price']='nan'

for v in ['car','bicycle','bike']:
    df_all['is_parking_'+v]=df_all[v+'_info'].apply(lambda x:add_is_park(x))
    df_all['is_house_parking_'+v]=df_all[v+'_info'].apply(lambda x:add_house_park(x))
    df_all['is_other_parking_'+v]=df_all[v+'_info'].apply(lambda x:add_other_park(x))
    df_all['parking_number_'+v]=df_all[v+'_info'].apply(lambda x:add_park_num(x))
    df_all['parking_price_'+v]=df_all[v+'_price'].apply(lambda x:add_park_price(x))

    
df_all=modify_outlier_price(df_all)

## 掲載している近隣情報の数を追加

In [111]:
df_all['neighbors']=df_all['neighbors'].fillna('nan')

def add_neighbor_info(df):
    neighbor_dict=[]
    convenience_count=[]
    supermarket_count=[]
    neighbor_count=[]
    neighbor_split=[]

    for i in tqdm(range(len(df))):
        tmp=dict()
        convenience=0
        supermarket=0

        neighbor_info=df['neighbors'][i]

        # 近隣情報が0 の場合は０を返す
        if neighbor_info=='nan':
            neighbor_dict.append('nan')
            convenience_count.append('nan')
            supermarket_count.append('nan')
            neighbor_count.append('nan')
            neighbor_split.append('nan')

        # 近隣情報の辞書配列を作成
        else:   
            for word in neighbor_info.split('\t'):
                place = re.findall(r'\【.+?\】', word)[0][1:-1]  # 建物の名前
                distance=int(re.findall(r'\d+m', word)[0][:-1])  #  距離

        # すでに同じ建物がある場合は、近い距離の値を入れる
                if place in tmp.keys():
                    tmp[place]=min(tmp[place],distance)
                else:
                    tmp[place]=distance

        #  コンビニとスーパー,近隣情報の合計の数をカウント
                if place=='コンビニ':
                    convenience+=1
                if place=='スーパー':
                    supermarket+=1
            neighbor=len(neighbor_info.split('\t'))
                
            neighbor_dict.append(tmp)
            convenience_count.append(convenience)
            supermarket_count.append(supermarket)
            neighbor_count.append(neighbor)
            neighbor_split.append(neighbor_info.split('\t'))

    df['neighbor_dict']=neightbor_dict
    df['convenience_count']=convenience_count
    df['supermarket_count']=supermarket_count
    df['neighbor_count']=neighbor_count
    df['neighbor_split']=neighbor_split
    
    return df

# 辞書から指定された場所の距離を取得
def neighbor_distance(x,place):
    if x==0:
        return 0
    elif place in x.keys():
        return x[place]
    else:
        return 0

def neighbor_encoder(df):
# 全ての場所の配列を取得
    places=[]
    for i in range(len(df)):
        neighbor=df['neighbor_dict'][i]
        if neighbor==0:
            continue
        else:
            places.extend(neighbor.keys())

    for place in set(places):
        df[place]=df['neighbor_dict'].apply(lambda x:neighbor_distance(x,place))

    return df

df_all=add_neighbor_info(df_all)
df_all=neighbor_encoder(df_all)

100%|██████████| 62723/62723 [00:01<00:00, 40164.23it/s]


In [112]:
df_all

,id,rent,address,access,floor_info,old,direction,square,floor,bath,kitchen,broadcast,facility,parking,neighbors,structure,period,neighbor_dict,convenience_count,supermarket_count,neighbor_count,neighbor_split,デパート,幼稚園・保育園,図書館,ドラッグストア,郵便局,大学,公園,スーパー,銀行,コインパーキング,レンタルビデオ,飲食店,小学校,コンビニ,病院,総合病院,月極駐車場,クリーニング,学校
0,1,75000.0,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間,"{'小学校': 962, '大学': 461, '公園': 1103, '飲食店': 378...",1,2,10,"[【小学校】 495m, 【大学】 461m, 【小学校】 962m, 【公園】 1103m...",0,0,0,956,1246,461,1103,1062,0,0,0,378,962,588,0,0,0,0,0
1,2,76000.0,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間,{'スーパー': 1283},0,1,1,[【スーパー】 1283m],0,0,0,0,0,0,0,1283,0,0,0,0,0,0,0,0,0,0,0
2,3,110000.0,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間,"{'スーパー': 492, 'コンビニ': 392}",2,2,4,"[【スーパー】 89m, 【コンビニ】 184m, 【コンビニ】 392m, 【スーパー】 ...",0,0,0,0,0,0,0,492,0,0,0,0,0,392,0,0,0,0,0
3,4,150000.0,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,"{'スーパー': 621, 'コンビニ': 328}",2,4,6,"[【スーパー】 225m, 【スーパー】 448m, 【スーパー】 619m, 【スーパー】...",0,0,0,0,0,0,0,621,0,0,0,0,0,328,0,0,0,0,0
4,5,74000.0,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間,"{'スーパー': 820, 'コンビニ': 351}",1,4,5,"[【スーパー】 193m, 【スーパー】 298m, 【スーパー】 660m, 【スーパー】...",0,0,0,0,0,0,0,820,0,0,0,0,0,351,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62718,62728,NaN,東京都豊島区上池袋４丁目,埼京線\t板橋駅\t徒歩3分\t\t都営三田線\t新板橋駅\t徒歩9分\t\t東武東上線\t...,1K,0年1ヶ月,東,25.66m2,6階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ／\t...,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【コンビニ】 204m\t【スーパー】 284m\t【スーパー】 496m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,"{'コンビニ': 577, 'スーパー': 714}",2,3,5,"[【コンビニ】 204m, 【スーパー】 284m, 【スーパー】 496m, 【コンビニ】...",0,0,0,0,0,0,0,714,0,0,0,0,0,577,0,0,0,0,0
62719,62729,NaN,東京都千代田区岩本町２丁目,都営新宿線\t岩本町駅\t徒歩2分\t\t山手線\t秋葉原駅\t徒歩4分\t\t日比谷線\t...,1R,15年6ヶ月,南,22.71m2,8階／15階建,専用バス／\t専用トイレ／\tシャワー／\t浴室乾燥機,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン,インターネット対応／\t高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ...,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気...,"駐車場\t空有\t43,200円\t駐輪場\t空有",nan,RC（鉄筋コンクリート）,NaN,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
62720,62730,NaN,東京都中野区江古田３丁目14-3,都営大江戸線\t新江古田駅\t徒歩10分\t\t西武池袋線\t江古田駅\t徒歩20分\t\t...,1LDK,1年0ヶ月,南東,45.76m2,10階／14階建,バス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座,ガスコンロ／\tコンロ3口／\t給湯,インターネット対応／\tCATV／\tインターネット使用料無料,エアコン付／\t床暖房\tフローリング／\t室内洗濯機置場／\t敷地内ごみ置き場\t／\tエ...,"駐車場\t空有\t32,400円(税込)\t駐輪場\t空有\tバイク置き場\t無",nan,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0

In [109]:
# 辞書から指定された場所の距離を取得
def neighbor_distance(x,place):
    if x==0:
        return 0
    elif place in x.keys():
        return x[place]
    else:
        return 0

# 全ての場所の配列を取得
places=[]
for i in range(len(df_all)):
    neighbor=df_all['neighbor_dict'][i]
    if neighbor==0:
        continue
    else:
        places.extend(neighbor.keys())

for place in set(places):
    df_all[place]=df_all['neighbor_dict'].apply(lambda x:neighbor_distance(x,place))

,id,rent,address,access,floor_info,old,direction,square,floor,bath,kitchen,broadcast,facility,parking,neighbors,structure,period,neighbor_dict,convenience_count,supermarket_count,neighbor_count,neighbor_split
0,1,75000.0,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,1K,9年9ヶ月,南東,20.01m2,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間,"{'小学校': 962, '大学': 461, '公園': 1103, '飲食店': 378...",1,2,10,"[【小学校】 495m, 【大学】 461m, 【小学校】 962m, 【公園】 1103m..."
1,2,76000.0,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,1R,44年10ヶ月,NaN,16.5m2,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間,{'スーパー': 1283},0,1,1,[【スーパー】 1283m]
2,3,110000.0,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,1K,8年6ヶ月,南,22.05m2,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間,"{'スーパー': 492, 'コンビニ': 392}",2,2,4,"[【スーパー】 89m, 【コンビニ】 184m, 【コンビニ】 392m, 【スーパー】 ..."
3,4,150000.0,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,2LDK,29年4ヶ月,南,60.48m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,"{'スーパー': 621, 'コンビニ': 328}",2,4,6,"[【スーパー】 225m, 【スーパー】 448m, 【スーパー】 619m, 【スーパー】..."
4,5,74000.0,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,2DK,31年7ヶ月,南,39.66m2,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間,"{'スーパー': 820, 'コンビニ': 351}",1,4,5,"[【スーパー】 193m, 【スーパー】 298m, 【スーパー】 660m, 【スーパー】..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62718,62728,NaN,東京都豊島区上池袋４丁目,埼京線\t板橋駅\t徒歩3分\t\t都営三田線\t新板橋駅\t徒歩9分\t\t東武東上線\t...,1K,0年1ヶ月,東,25.66m2,6階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCATV／\tCSアンテナ／\tBSアンテナ／\t...,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【コンビニ】 204m\t【スーパー】 284m\t【スーパー】 496m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間,"{'コンビニ': 577, 'スーパー': 714}",2,3,5,"[【コンビニ】 204m, 【スーパー】 284m, 【スーパー】 496m, 【コンビニ】..."
62719,62729,NaN,東京都千代田区岩本町２丁目,都営新宿線\t岩本町駅\t徒歩2分\t\t山手線\t秋葉原駅\t徒歩4分\t\t日比谷線\t...,1R,15年6ヶ月,南,22.71m2,8階／15階建,専用バス／\t専用トイレ／\tシャワー／\t浴室乾燥機,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t独立キッチン,インターネット対応／\t高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ...,冷房／\tエアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t24時間換気...,"駐車場\t空有\t43,200円\t駐輪場\t空有",nan,RC（鉄筋コンクリート）,NaN,0,nan,nan,nan,nan
62720,62730,NaN,東京都中野区江古田３丁目14-3,都営大江戸線\t新江古田駅\t徒歩10分\t\t西武池袋線\t江古田駅\t徒歩20分\t\t...,1LDK,1年0ヶ月,南東,45.76m2,10階／14階建,バス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座,ガスコンロ／\tコンロ3口／\t給湯,インターネット対応／\tCATV／\tインターネット使用料無料,エアコン付／\t床暖房\tフローリング／\t室内洗濯機置場／\t敷地内ごみ置き場\t／\tエ...,"駐車場\t空有\t32,400円(税込)\t駐輪場\t空有\tバイク置き場\t無",nan,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,0,nan,nan,nan,nan
62721,62731,NaN,東京都千代田区二番町,有楽町線\t麹町駅\t徒歩3分\t\t丸ノ内線(池袋－荻窪)\t四ツ谷駅\t徒歩4分\t\t...,1K,15年4ヶ月,北,55.2m2,14階／14階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,IHコンロ\t／\tコンロ3口／\tシステムキッチン\t／\t独立キッチン,インターネット対応／\t高速インターネット／\t光ファイバー／\tCATV／\tCSアンテナ...,冷房／\tエアコン付／\t床暖房\tバルコニー／\tフローリング／\t24時間換気システム\...,駐輪場\t空有,nan,SRC（鉄骨鉄筋コンクリート）,NaN,0,nan,nan,nan,nan


In [110]:
for i in range(10,20):
    print(df_all['neighbor_split'][i])
    print(df_all['neighbor_count'][i])

['【スーパー】 406m', '【スーパー】 406m', '【スーパー】 483m', '【コンビニ】 326m', '【コンビニ】 417m']
5
nan
nan
nan
nan
nan
nan
['【ドラッグストア】 612m', '【コンビニ】 773m', '【スーパー】 887m', '【飲食店】 791m']
4
['【スーパー】 318m', '【スーパー】 564m', '【スーパー】 477m', '【スーパー】 544m']
4
['【コンビニ】 122m', '【スーパー】 119m', '【ドラッグストア】 225m', '【病院】 530m', '【郵便局】 221m', '【図書館】 755m', '【銀行】 277m', '【公園】 362m']
8
['【小学校】 523m', '【学校】 46m', '【幼稚園・保育園】 136m', '【郵便局】 537m', '【図書館】 1219m', '【公園】 506m', '【病院】 1299m']
7
['【スーパー】 228m', '【スーパー】 714m', '【コンビニ】 569m', '【コンビニ】 1103m', '【郵便局】 1038m', '【総合病院】 1364m']
6
['【スーパー】 237m', '【スーパー】 581m', '【コンビニ】 79m', '【コンビニ】 196m']
4


In [79]:
for i in df_all['neighbors'].unique():
    print(i)

【小学校】 495m	【大学】 461m	【小学校】 962m	【公園】 1103m	【飲食店】 378m	【スーパー】 311m	【コンビニ】 588m	【スーパー】 1062m	【ドラッグストア】 956m	【郵便局】 1246m
【スーパー】 1283m
【スーパー】 89m	【コンビニ】 184m	【コンビニ】 392m	【スーパー】 492m
【スーパー】 225m	【スーパー】 448m	【スーパー】 619m	【スーパー】 621m	【コンビニ】 328m	【コンビニ】 326m
【スーパー】 193m	【スーパー】 298m	【スーパー】 660m	【スーパー】 820m	【コンビニ】 351m
【スーパー】 251m	【コンビニ】 376m
【スーパー】 812m	【コンビニ】 149m	【コンビニ】 942m	【スーパー】 1300m	【コンビニ】 813m
【スーパー】 334m	【スーパー】 562m	【スーパー】 694m	【コンビニ】 344m	【コンビニ】 567m	【コンビニ】 782m
0
【スーパー】 406m	【スーパー】 406m	【スーパー】 483m	【コンビニ】 326m	【コンビニ】 417m
【ドラッグストア】 612m	【コンビニ】 773m	【スーパー】 887m	【飲食店】 791m
【スーパー】 318m	【スーパー】 564m	【スーパー】 477m	【スーパー】 544m
【コンビニ】 122m	【スーパー】 119m	【ドラッグストア】 225m	【病院】 530m	【郵便局】 221m	【図書館】 755m	【銀行】 277m	【公園】 362m
【小学校】 523m	【学校】 46m	【幼稚園・保育園】 136m	【郵便局】 537m	【図書館】 1219m	【公園】 506m	【病院】 1299m
【スーパー】 228m	【スーパー】 714m	【コンビニ】 569m	【コンビニ】 1103m	【郵便局】 1038m	【総合病院】 1364m
【スーパー】 237m	【スーパー】 581m	【コンビニ】 79m	【コンビニ】 196m
【小学校】 179m	【大学】 470m	【郵便局】 594m	【小学校】 1013m	【公園】 1089m	【公園】 1209m	【スーパー】 678m	【スーパ

【スーパー】 483m	【コンビニ】 465m	【ドラッグストア】 421m	【総合病院】 967m	【郵便局】 941m	【レンタルビデオ】 197m
【コンビニ】 250m	【スーパー】 759m	【コンビニ】 272m	【コンビニ】 349m	【コンビニ】 552m	【コンビニ】 794m	【公園】 1500m
【スーパー】 944m	【コンビニ】 393m	【小学校】 563m	【病院】 334m	【公園】 765m
【スーパー】 411m	【スーパー】 550m	【スーパー】 662m	【コンビニ】 435m	【コンビニ】 480m
【スーパー】 384m	【スーパー】 775m	【スーパー】 838m	【コンビニ】 191m	【コンビニ】 319m	【コンビニ】 315m
【小学校】 1240m	【スーパー】 767m	【スーパー】 957m
【スーパー】 623m	【スーパー】 928m	【コンビニ】 166m	【病院】 1095m
【スーパー】 242m	【スーパー】 829m	【スーパー】 660m
【スーパー】 300m	【スーパー】 487m	【コンビニ】 196m	【コンビニ】 331m	【公園】 2300m
【デパート】 875m	【レンタルビデオ】 1000m	【飲食店】 937m	【大学】 1024m	【公園】 1045m	【コンビニ】 1153m
【スーパー】 570m	【スーパー】 675m	【コンビニ】 463m	【コンビニ】 465m
【スーパー】 411m	【スーパー】 1095m	【コンビニ】 582m	【コンビニ】 603m	【コンビニ】 804m	【公園】 810m
【スーパー】 304m	【スーパー】 487m	【スーパー】 662m	【コンビニ】 276m	【コンビニ】 503m	【コンビニ】 577m
【スーパー】 683m	【スーパー】 966m
【小学校】 500m	【学校】 300m	【コンビニ】 300m	【スーパー】 750m	【公園】 500m	【郵便局】 250m
【小学校】 602m	【学校】 889m	【幼稚園・保育園】 379m	【郵便局】 59m	【図書館】 1331m	【公園】 1327m	【病院】 321m
【スーパー】 423m	【コンビニ】 672m	【スーパー】 1076m	【スーパ

【スーパー】 315m	【スーパー】 347m	【コンビニ】 222m	【コンビニ】 382m	【ドラッグストア】 383m	【クリーニング】 440m	【銀行】 348m	【飲食店】 518m	【飲食店】 420m
【スーパー】 218m	【スーパー】 622m	【スーパー】 980m	【コンビニ】 782m	【コンビニ】 783m
【スーパー】 915m	【スーパー】 433m	【スーパー】 866m	【コンビニ】 324m
【スーパー】 453m	【スーパー】 453m	【スーパー】 546m	【スーパー】 480m	【スーパー】 713m	【スーパー】 771m
【スーパー】 128m	【コンビニ】 412m	【スーパー】 1016m	【コンビニ】 492m	【コンビニ】 561m	【飲食店】 1508m	【スーパー】 252m	【総合病院】 507m	【飲食店】 1139m
【小学校】 368m	【スーパー】 154m	【コンビニ】 206m	【公園】 112m	【郵便局】 163m	【銀行】 1274m
【スーパー】 567m	【飲食店】 861m	【スーパー】 775m	【スーパー】 1006m
【スーパー】 404m
【ドラッグストア】 314m	【ドラッグストア】 355m	【飲食店】 281m	【飲食店】 289m	【スーパー】 42m	【スーパー】 232m	【コンビニ】 131m	【コンビニ】 236m	【郵便局】 330m	【銀行】 314m
【スーパー】 1255m	【スーパー】 1345m	【コンビニ】 479m	【コンビニ】 587m
【スーパー】 291m	【スーパー】 686m	【スーパー】 832m	【スーパー】 1011m	【コンビニ】 538m	【ドラッグストア】 1197m	【郵便局】 1193m	【銀行】 729m	【銀行】 954m
【スーパー】 470m	【コンビニ】 89m	【コンビニ】 130m	【ドラッグストア】 214m	【総合病院】 569m	【学校】 815m	【幼稚園・保育園】 1122m	【飲食店】 713m
【スーパー】 81m	【コンビニ】 182m
【コンビニ】 200m	【コンビニ】 221m	【コンビニ】 323m	【スーパー】 273m	【スーパー】 571m
【スーパー】 160m	【ス

【スーパー】 883m	【コンビニ】 682m	【コンビニ】 805m	【コンビニ】 1003m	【コンビニ】 1069m
【スーパー】 848m	【スーパー】 753m	【スーパー】 737m	【コンビニ】 270m	【コンビニ】 469m	【コンビニ】 884m
【図書館】 1490m	【公園】 51m	【コンビニ】 123m	【コンビニ】 240m	【コンビニ】 295m	【スーパー】 234m	【スーパー】 250m
【コンビニ】 303m	【小学校】 562m	【総合病院】 93m	【公園】 580m
【コンビニ】 230m	【スーパー】 340m	【ドラッグストア】 410m	【銀行】 460m
【スーパー】 472m	【スーパー】 454m	【スーパー】 778m	【コンビニ】 136m	【コンビニ】 245m	【総合病院】 508m	【総合病院】 1130m	【病院】 634m	【病院】 1023m
【スーパー】 250m	【スーパー】 574m	【スーパー】 927m	【コンビニ】 253m	【コンビニ】 240m
【公園】 207m	【コンビニ】 370m	【スーパー】 485m	【総合病院】 529m	【小学校】 657m
【スーパー】 348m	【スーパー】 421m	【コンビニ】 150m	【コンビニ】 250m
【飲食店】 608m	【飲食店】 577m	【スーパー】 224m	【スーパー】 483m	【スーパー】 458m	【コンビニ】 91m	【コンビニ】 195m	【ドラッグストア】 232m	【コンビニ】 273m
【スーパー】 115m	【ドラッグストア】 180m	【コンビニ】 255m	【スーパー】 427m	【コンビニ】 354m	【図書館】 192m
【小学校】 499m	【学校】 491m	【幼稚園・保育園】 147m	【郵便局】 127m	【図書館】 393m	【公園】 1223m	【病院】 37m
【コンビニ】 288m	【スーパー】 651m
【スーパー】 187m	【スーパー】 398m	【コンビニ】 109m	【コンビニ】 124m	【ドラッグストア】 349m
【スーパー】 260m	【コンビニ】 230m	【病院】 150m	【スーパー】 162m
【スーパー】 854m	【スーパー】 934m	【スーパ

【スーパー】 80m	【コンビニ】 249m	【病院】 953m
【コンビニ】 202m	【スーパー】 743m	【総合病院】 251m	【小学校】 610m	【公園】 333m
【小学校】 350m	【学校】 1100m	【コンビニ】 41m	【スーパー】 140m	【郵便局】 350m	【公園】 260m
【スーパー】 442m	【コンビニ】 86m	【ドラッグストア】 460m	【総合病院】 286m
【スーパー】 594m	【コンビニ】 579m	【スーパー】 794m	【スーパー】 460m
【コンビニ】 221m	【スーパー】 338m	【総合病院】 215m	【公園】 405m	【小学校】 809m
【スーパー】 578m	【スーパー】 687m	【コンビニ】 273m	【コンビニ】 333m	【病院】 2585m
【スーパー】 417m	【スーパー】 759m	【コンビニ】 407m	【コンビニ】 491m
【ドラッグストア】 86m	【銀行】 167m	【総合病院】 189m	【飲食店】 335m	【スーパー】 293m	【デパート】 346m	【レンタルビデオ】 379m
【スーパー】 142m	【コンビニ】 56m
【スーパー】 197m	【スーパー】 456m	【コンビニ】 300m	【コンビニ】 312m	【幼稚園・保育園】 452m
【ドラッグストア】 62m	【飲食店】 101m	【スーパー】 177m	【コンビニ】 32m	【郵便局】 337m	【公園】 924m	【図書館】 489m	【病院】 574m	【大学】 810m
【スーパー】 272m	【飲食店】 591m	【飲食店】 490m	【飲食店】 616m	【飲食店】 465m	【飲食店】 1011m	【スーパー】 593m	【コンビニ】 683m
【スーパー】 573m	【スーパー】 482m	【スーパー】 615m	【コンビニ】 636m
【スーパー】 233m	【スーパー】 424m	【スーパー】 452m	【コンビニ】 97m	【コンビニ】 501m	【幼稚園・保育園】 268m
【総合病院】 1511m	【スーパー】 167m	【スーパー】 181m	【コンビニ】 206m	【コンビニ】 327m	【ドラッグストア】 383m
【スーパー】 237m	【コンビニ】 2

...

【スーパー】 287m	【スーパー】 572m	【スーパー】 631m	【コンビニ】 356m	【コンビニ】 450m	【コンビニ】 488m	【クリーニング】 415m	【郵便局】 482m	【飲食店】 636m	【飲食店】 987m
【小学校】 450m	【学校】 1100m	【スーパー】 400m	【公園】 500m
【小学校】 469m	【スーパー】 53m	【スーパー】 191m	【スーパー】 202m	【コンビニ】 127m	【病院】 143m	【図書館】 487m
【小学校】 350m	【学校】 800m	【コンビニ】 210m	【スーパー】 170m	【公園】 1100m	【ドラッグストア】 400m
【スーパー】 572m	【コンビニ】 219m	【病院】 651m
【小学校】 844m	【学校】 562m	【幼稚園・保育園】 366m	【郵便局】 626m	【図書館】 662m	【公園】 211m	【病院】 1040m
【幼稚園・保育園】 423m	【幼稚園・保育園】 239m	【小学校】 528m	【学校】 608m	【飲食店】 778m	【飲食店】 588m	【飲食店】 621m	【スーパー】 628m	【コンビニ】 301m
【スーパー】 146m	【スーパー】 327m	【コンビニ】 281m
【スーパー】 112m	【コンビニ】 121m	【病院】 348m	【郵便局】 1135m
【スーパー】 371m	【コンビニ】 349m
【ドラッグストア】 396m	【コンビニ】 507m	【スーパー】 511m	【スーパー】 512m	【郵便局】 652m	【銀行】 569m
【小学校】 1076m	【学校】 1051m	【幼稚園・保育園】 470m	【郵便局】 530m	【図書館】 717m	【公園】 1408m	【病院】 962m
【スーパー】 250m	【コンビニ】 100m	【ドラッグストア】 400m	【スーパー】 420m	【コンビニ】 330m
【スーパー】 489m	【コンビニ】 527m	【小学校】 272m
【小学校】 173m	【学校】 642m	【幼稚園・保育園】 104m	【郵便局】 404m	【図書館】 2094m	【公園】 777m	【病院】 1262m
【コンビニ】 350m	【スーパー】 1m	【銀行】 1m

【スーパー】 688m	【スーパー】 886m	【コンビニ】 847m	【コンビニ】 651m
【コンビニ】 234m	【スーパー】 361m	【小学校】 543m	【総合病院】 726m	【公園】 775m
【スーパー】 405m	【コンビニ】 353m	【小学校】 176m	【総合病院】 151m	【公園】 214m
【小学校】 350m	【学校】 1000m	【スーパー】 500m	【コンビニ】 650m	【ドラッグストア】 450m	【公園】 25m
【学校】 629m	【大学】 812m	【大学】 1143m	【飲食店】 925m	【飲食店】 945m	【飲食店】 930m	【飲食店】 861m	【飲食店】 1352m
【コンビニ】 330m	【銀行】 350m	【銀行】 360m	【コンビニ】 522m	【郵便局】 542m	【コンビニ】 553m	【飲食店】 577m
【大学】 989m	【大学】 1340m	【大学】 1617m	【公園】 756m	【スーパー】 451m	【スーパー】 644m	【スーパー】 1049m
【スーパー】 629m	【コンビニ】 820m	【コンビニ】 940m
【スーパー】 987m	【スーパー】 1070m	【スーパー】 1087m	【コンビニ】 1055m	【コンビニ】 1041m	【飲食店】 384m	【クリーニング】 918m	【郵便局】 1002m	【郵便局】 1067m	【飲食店】 976m
【学校】 1665m	【飲食店】 938m	【飲食店】 971m	【飲食店】 1441m	【飲食店】 1436m	【スーパー】 723m	【スーパー】 1122m	【コンビニ】 1037m	【コンビニ】 1431m	【ドラッグストア】 1088m
【スーパー】 606m	【総合病院】 1007m	【総合病院】 1585m	【幼稚園・保育園】 1716m	【スーパー】 1763m	【飲食店】 1926m
【スーパー】 770m	【スーパー】 1651m	【スーパー】 2077m	【コンビニ】 717m	【コンビニ】 1108m	【コンビニ】 1530m
【コンビニ】 201m	【スーパー】 59m	【ドラッグストア】 123m	【スーパー】 125m	【コンビニ】 140m	【スーパー】 118m	【銀行】 304m
【

【スーパー】 300m	【スーパー】 360m	【スーパー】 896m	【コンビニ】 402m	【コンビニ】 533m	【公園】 80m
【スーパー】 565m	【スーパー】 596m	【スーパー】 619m	【コンビニ】 565m	【コンビニ】 682m	【コンビニ】 788m
【郵便局】 389m	【コンビニ】 309m	【コンビニ】 295m	【コンビニ】 164m	【スーパー】 359m	【スーパー】 256m	【スーパー】 108m	【飲食店】 398m	【飲食店】 352m	【ドラッグストア】 302m
【コンビニ】 118m
【スーパー】 307m	【コンビニ】 502m	【コンビニ】 579m
【スーパー】 71m	【スーパー】 654m	【スーパー】 618m	【スーパー】 682m	【スーパー】 698m	【スーパー】 764m
【スーパー】 812m	【スーパー】 737m	【コンビニ】 875m	【コンビニ】 911m	【病院】 2899m
【スーパー】 165m	【コンビニ】 252m	【小学校】 720m	【総合病院】 598m	【幼稚園・保育園】 624m
【コンビニ】 260m	【コンビニ】 281m	【スーパー】 289m	【スーパー】 512m	【スーパー】 621m	【スーパー】 743m	【公園】 610m
【飲食店】 215m	【スーパー】 268m	【スーパー】 295m	【スーパー】 359m	【コンビニ】 178m	【コンビニ】 236m	【郵便局】 200m	【ドラッグストア】 237m	【飲食店】 196m	【飲食店】 261m
【スーパー】 474m	【スーパー】 670m	【スーパー】 679m	【飲食店】 592m	【飲食店】 504m	【コンビニ】 841m
【大学】 951m	【大学】 1188m	【大学】 2353m	【スーパー】 687m	【コンビニ】 533m	【総合病院】 1118m	【総合病院】 4443m	【総合病院】 4576m
【スーパー】 464m	【コンビニ】 653m
【大学】 422m	【大学】 1305m	【大学】 1352m	【郵便局】 333m	【銀行】 802m
【スーパー】 425m	【スーパー】 897m	【コンビニ】 341m	【総合病院】 1193m	【レンタルビデ

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



【スーパー】 186m	【ドラッグストア】 308m	【コンビニ】 278m	【ドラッグストア】 353m	【スーパー】 373m	【ドラッグストア】 432m
【スーパー】 516m	【コンビニ】 243m	【ドラッグストア】 542m	【郵便局】 502m	【病院】 474m	【銀行】 491m
【スーパー】 38m	【スーパー】 263m	【コンビニ】 257m	【ドラッグストア】 34m	【ドラッグストア】 227m
【コンビニ】 350m	【スーパー】 650m	【スーパー】 1500m
【スーパー】 324m	【スーパー】 601m	【コンビニ】 289m	【コンビニ】 248m	【病院】 804m
【コンビニ】 1092m	【コンビニ】 986m
【スーパー】 274m	【スーパー】 654m	【コンビニ】 196m	【コンビニ】 414m
【大学】 561m	【飲食店】 709m	【スーパー】 580m	【コンビニ】 682m	【ドラッグストア】 704m	【飲食店】 679m	【飲食店】 631m	【大学】 802m	【飲食店】 891m
【飲食店】 907m	【飲食店】 442m	【飲食店】 1567m	【飲食店】 1581m	【飲食店】 345m	【飲食店】 1586m	【デパート】 1734m	【スーパー】 228m	【スーパー】 268m
【スーパー】 490m	【スーパー】 581m	【スーパー】 481m	【コンビニ】 508m	【スーパー】 611m	【総合病院】 610m
【小学校】 297m	【学校】 794m	【幼稚園・保育園】 534m	【郵便局】 238m	【図書館】 644m	【公園】 968m	【病院】 846m
【総合病院】 111m	【幼稚園・保育園】 273m	【コンビニ】 175m	【コンビニ】 359m	【スーパー】 452m	【コンビニ】 448m
【スーパー】 202m	【飲食店】 690m	【総合病院】 677m	【郵便局】 841m	【幼稚園・保育園】 847m	【コンビニ】 813m
【小学校】 329m	【学校】 611m	【幼稚園・保育園】 396m	【郵便局】 374m	【図書館】 475m	【公園】 1538m	【病院】 1076m
【スーパー】 754m	【コンビニ】 177m	【スーパー】